##  概念

正则化是一种旨在减少泛化误差而不是训练误差的方法，是机器学习中非常重要并且有效的技术，特别是在深度学习模型中，由于其模型参数非常多非常容易产生过拟合。因此需要引入正则化来抑制模型过拟合现象，提高模型泛化能力，常用的正则化方法有：  

1 参数添加约束，例如L1、L2范数等  

2 训练集合扩充，例如添加噪声、数据变换等


## 1 L2 正则化  

L2 正则化是最常用的正则化方法。可以通过惩罚目标函数中所有参数的平方将其实现。即对于网络中的每个权重 w，向目标函数中增加一个 $\frac{1}{2}\lambda w^2$，其中 $\lambda$ 是正则化强度。前面这个 $\frac{1}{2}$ 很常见，是因为加上 $\frac{1}{2}$ 后，该式子关于 w 梯度就是 $\lambda w$ 而不是 $2\lambda w$ 了。L2 正则化可以直观理解为它对于大数值的权重向量进行严厉惩罚，倾向于更加分散的权重向量,这样就有了一个优良的特性：使网络更倾向于使用所有输入特征，而不是严重依赖输入特征中某些小部分特征。最后需要注意在梯度下降和参数更新的时候，使用 L2 正则化意味着所有的权重都以 $w += -\lambda * W$ 向着 0 线性下降。

## 2 L1正则化  

L1 正则化是另一个相对常用的正则化方法。对于每个 w 我们都向目标函数增加一个$\lambda|w|$。L1 和 L2 正则化也可以进行组合：
$\lambda_1|w|+\lambda_2w^2$，这也被称作 Elastic net regularizaton。L1 正则化有一个有趣的性质，它会让权重向量在最优化的过程中变得稀疏（即非常接近0）。也就是说，使用 L1 正则化的神经元最后使用的是它们最重要的输入数据的稀疏子集，同时对于噪音输入则几乎是不变的了。相较 L1 正则化，L2 正则化中的权重向量大多是分散的小数字。在实践中，如果不是特别关注某些明确的特征选择，一般说来 L2 正则化都会比 L1 正则化效果好。

L2 和 L1 正则具有不同的性质：L2得到平滑的权值， L1得到稀疏的权值  

当几个特征的共线性很高，并且对分类都很重要，L2 倾向于将权值平分给这些特征，从而使得这些有用的特征都得以留下；L1 则会随机选择其中一个特征，而扔掉其它的特征，所以说L2平滑，L1稀疏。下面从两个方面解释这两个性质。  

### 2.1 数学推导  

首先来看 L2 和 L1 的梯度的更新方式：

$L2 = \frac {1}{2}(w_1^2 + w_2^2 + ... + w_n^2), \ \ \ \ \frac {\partial L_2}{w_i} = w_i$  

$L1 = |w_1| + |w_2| + ... + |w_n| , \ \ \ \ \frac {\partial L_1}{w_i} = sign(w_i) = 1 \ or \  -1$  

L2 的权值更新公式为 $w_i = w_i - \eta * w_i$，可见权值不断变小，但是因为每次都减少上一次的 $\eta$ 倍，在小数值时候，权值下降的较慢，很难下降到 0，会收敛到较小但不为 0 的值，所以训练好的权值比较平滑。


L1 的权值更新公式为 $w_i = w_i - \eta * sign(w_i)$，也就是说权值每次更新都固定减少一个特定的值 $\eta$，在小数值时，权值下降较快，经过若干次迭代之后，权值有可能减少到 0。  

### 2.2 几何空间  

下图是 PRML 中 介绍 L2 与 L1 不同性质的图：  


这里刚开始看不明白，优化的目标函数应该是 $\min\limits_w E_D(w) + \lambda E_R(w)$, 损失函数包含了两部分，应该结合起来进行优化，但图里面将两项分开进行优化了，这里进行简单的解释：  

$\min\limits_w E_D(w) + \lambda E_R(w)$ 和 $\begin{cases} \min\limits_w E_D(w) \\ s.t.E_R(w) \leq \eta \end{cases}$ 是等价的，即对一个特定的 $\lambda$ 总存在一个 $\eta$ 使得这两个问题是等价的。

上图表达的是对目标函数的等价变换之后的形式，把 w 的解限制在黄色区域内，同时使得经验损失尽可能小。  

为了便于可视化，上图考虑两维的情况，在(w1,w2)平面上可以画出目标函数的等高线，而约束条件则成为平面上半径为 $\eta$ 的一个 norm ball。等高线与 norm ball 首次相交的地方就是最优解：  
 
不同的同心圆就是优化过程中遇到的等高线，可以看到，L1-ball 与 L2-ball 的不同就在于 L1 在和每个坐标轴相交的地方都有“角”出现，除非目标函数比较特殊(位于特定的位置)，否则大部分条件下都会在角的地方相交，角的位置就会产生稀疏性。

相比之下，L2-ball 就没有这样的性质，因为没有角，所以第一次相交的地方出现在具有稀疏性的位置的概率就变得非常小了，所以 L2 计算出来的权重就比较平滑。

## 3 Dropout  

随机失活（Dropout）是一个简单又极其有效的正则化方法。该方法由 Srivastava 在论文 Dropout: A Simple Way to Prevent Neural Networks from Overfitting 中提出的，与 L1 正则化，L2 正则化和最大范式约束等方法互为补充。在训练的时候，随机失活的实现方法是让神经元以超参数 p 的概率被激活或者被设置为0。  

在训练过程中，随机失活可以被认为是对完整的神经网络抽样出一些子集，每次基于输入数据只更新子网络的参数。在测试过程中不使用随机失活，可以理解为是对数量巨大的子网络们做了模型集成，以此来计算出一个平均的预测。